In [54]:
import datasets
from tqdm.notebook import tqdm
import pandas as pd
import time

In [2]:
fnc_stance_detection = pd.read_csv("../data/fnc_stance_detection/train_bodies.csv")

In [15]:
len(fnc_stance_detection)

1683

In [3]:
fnc_stance_detection.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [4]:
fnc_stance_detection_stances = pd.read_csv("../data/fnc_stance_detection/train_stances.csv")

In [16]:
len(fnc_stance_detection_stances)

49972

In [5]:
fnc_stance_detection_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [67]:
data = {}
stances_counts = set()
for i, e in tqdm(fnc_stance_detection.iterrows(), total=len(fnc_stance_detection)):
    body_id = e['Body ID']
    article_body = e['articleBody']
    headlines = []
    stances = fnc_stance_detection_stances[fnc_stance_detection_stances['Body ID'] == body_id]
    for j, stance in stances.iterrows():
        headlines.append((stance['Headline'], stance['Stance']))
    data[body_id] = {"article_body": article_body, "headlines": headlines}
    stances_counts.add(len(headlines))

  0%|          | 0/1683 [00:00<?, ?it/s]

In [42]:
data[158]

{'article_body': "Hundreds of Palestinians were evacuated from their homes Sunday morning after Israeli authorities opened a number of dams near the border, flooding the Gaza Valley in the wake of a recent severe winter storm.\n\nThe Gaza Ministry of Interior said in a statement that civil defense services and teams from the Ministry of Public Works had evacuated more than 80 families from both sides of the Gaza Valley (Wadi Gaza) after their homes flooded as water levels reached more than three meters.\n\nGaza has experienced flooding in recent days amid a major storm that saw temperatures drop and frigid rain pour down.\n\nThe storm displaced dozens and caused hardship for tens of thousands, including many of the approximately 110,000 Palestinians left homeless by Israel's assault over summer.\n\nThe suffering is compounded by the fact that Israel has maintained a complete siege over Gaza for the last eight years, severely limiting electricity and the availability of fuel for generat

In [27]:
fnc_stance_detection_stances[fnc_stance_detection_stances['Body ID'] == 158]

,Headline,Body ID,Stance
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
3107,It's 'rubbish' that Robert Plant turned down £...,158,unrelated
6392,Robert Plant ripped up $800M Led Zeppelin reun...,158,unrelated
8059,ISIS Militant “Jihadi John” Identified As Youn...,158,unrelated
11688,Claim: Comcast Got Complaining Customer Fired ...,158,unrelated
20618,Jordan king cites Clint Eastwood in revenge vow,158,unrelated
22871,7 Bosnian teens get pregnant on school trip,158,unrelated
23346,Robert Plant Reportedly Tears Up $800 Million ...,158,unrelated
23849,"No, Robert Plant Didn’t Rip Up an $800 Million...",158,unrelated
24199,Led Zeppelin Reunion: Robert Plant Rips Up $30...,158,unrelated


In [26]:
fnc_stance_detection[fnc_stance_detection['Body ID'] == 158]['articleBody'].values[0]

"Hundreds of Palestinians were evacuated from their homes Sunday morning after Israeli authorities opened a number of dams near the border, flooding the Gaza Valley in the wake of a recent severe winter storm.\n\nThe Gaza Ministry of Interior said in a statement that civil defense services and teams from the Ministry of Public Works had evacuated more than 80 families from both sides of the Gaza Valley (Wadi Gaza) after their homes flooded as water levels reached more than three meters.\n\nGaza has experienced flooding in recent days amid a major storm that saw temperatures drop and frigid rain pour down.\n\nThe storm displaced dozens and caused hardship for tens of thousands, including many of the approximately 110,000 Palestinians left homeless by Israel's assault over summer.\n\nThe suffering is compounded by the fact that Israel has maintained a complete siege over Gaza for the last eight years, severely limiting electricity and the availability of fuel for generators. It has also 

In [19]:
fnc_stance_detection_stances[fnc_stance_detection_stances['Stance'] == "agree"]['Headline'][1]

'Hundreds of Palestinians flee floods in Gaza as Israel opens dams'

In [6]:
fnc_stance_detection_stances[fnc_stance_detection_stances['Body ID'] == 158].head()

,Headline,Body ID,Stance
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
3107,It's 'rubbish' that Robert Plant turned down £...,158,unrelated
6392,Robert Plant ripped up $800M Led Zeppelin reun...,158,unrelated
8059,ISIS Militant “Jihadi John” Identified As Youn...,158,unrelated
11688,Claim: Comcast Got Complaining Customer Fired ...,158,unrelated


In [8]:
import ollama

In [10]:
! ollama pull llama3.1

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
ollama.pull('llama3.1')

{'status': 'success'}

In [14]:
response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

A classic question!

The short answer is: because of the way that light scatters in the Earth's atmosphere. Here's a more detailed explanation:

When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but the shorter (blue) wavelengths are scattered more than the longer (red) wavelengths. This phenomenon is called Rayleigh scattering, named after the British physicist Lord Rayleigh who first described it in 1874.

As a result of this scattering, the blue light becomes distributed throughout the atmosphere and reaches our eyes from all directions, giving the sky its blue appearance during the daytime. The exact shade of blue can vary depending on factors such as atmospheric conditions, pollution levels, and time of day.

So, to summarize: the sky appears blue because of the way that sunlight scatters in the Earth's atmosphere!


In [56]:
prompt = """Here you are given a text (between <text> tags), as well as a short headline, the headline (between <headline> tags) will either agree or disagree with the content of the text, or be completely random and unrelated.

<text> {text} </text>

<headline> {headline} </headline>

Output just one of the following words: agree, disagree, or unrelated. The answer is: """

In [57]:
print(prompt.format(text='Hey', headline="hello"))

Here you are given a text, as well as a short headline, the headline will either agree or disagree with the content of the text, or be completely random and unrelated.

### beginning of text ###
Hey
### end of text ###

### beginning of headline ###
hello
### end of headline ###

Output just one of the following words: agree, disagree, or unrelated. The answer is: 


In [59]:
t = time.time()
answers = {}
unique_outputs = set()
for i, d in tqdm(data.items()):
    text = d['article_body']
    answers[i] = []
    for headline, label in d['headlines']:
        p = prompt.format(text=text, headline=headline)
        response = ollama.chat(model='llama3.1', messages=[
          {
            'role': 'user',
            'content': p,
          },
        ])
        pred = response['message']['content']
        answers[i].append((pred, label))
        unique_outputs.add(pred)
    
    if len(answers) == 10:
        break
    
    
print("Elapsed time: ", time.time() - t)

  0%|          | 0/1683 [00:00<?, ?it/s]

Elapsed time:  174.1006202697754


In [64]:
acc = []
extra = []
for i, dd in answers.items():
    for d in dd:
        pred, label = d
        pred = pred.split('.')[0].split('\n')[0].lower()
        acc.append(int(pred == label))
        if pred not in ['agree', 'disagree', 'unrelated']:
            extra.append(pred)

print('Average acc:', sum(acc)/len(acc))
print(extra)

Average acc: 0.5786666666666667
['disagreeassistant', 'disagreeassistant', 'disagreeassistant', 'agreeassistant', 'disagree ']


In [65]:
prompt = """Here you are given a text (between <text> tags), as well as a short headline, the headline (between <headline> tags) will either agree or disagree with the content of the text, or be completely random and unrelated.

<text> {text} </text>

<headline> {headline} </headline>

Output just one of the following words: agree, disagree, or unrelated. The answer is: """

In [68]:
t = time.time()
answers = {}
unique_outputs = set()
for i, d in tqdm(data.items()):
    text = d['article_body']
    answers[i] = []
    for headline, label in d['headlines']:
        p = prompt.format(text=text, headline=headline)
        response = ollama.chat(model='llama3.1', messages=[
          {
            'role': 'user',
            'content': p,
          },
        ])
        pred = response['message']['content']
        answers[i].append((pred, label))
        unique_outputs.add(pred)
    
    if len(answers) == 10:
        break
    
    
print("Elapsed time: ", time.time() - t)

  0%|          | 0/1683 [00:00<?, ?it/s]

Elapsed time:  315.48152232170105


In [72]:
acc = []
extra = []
for i, dd in answers.items():
    for d in dd:
        pred, label = d
        pred = pred.split('.')[0].split(' ')[0].split('\n')[0].lower()
        print(pred, label)
        acc.append(int(pred == label))
        if pred not in ['agree', 'disagree', 'unrelated']:
            extra.append(pred)

print('Average acc:', sum(acc)/len(acc))
print(extra)

disagree unrelated
disagree unrelated
agree unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
agree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
unrelated unrelated
agree unrelated
agree unrelated
agree unrelated
agree unrelated
disagree unrelated
disagree unrelated
unrelated unrelated
disagree unrelated
agree agree
i unrelated
unrelated unrelated
disagree unrelated
agree unrelated
unrelated unrelated
unrelated unrelated
agree unrelated
agree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree discuss
disagree unrelated
disagree unrelated
unrelated unrelated
unrelated unrelated
agree unrelated
agree unrelated
agree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
disagree unrelated
unrelated unrelated
"agree" unrelated
agree unrelated
agree unrelated
disagree unrelated
agree unrelated
disagree unrelated
disagr

In [73]:
prompt = """Here you are given a text (between <text> tags), as well as a short headline, the headline (between <headline> tags) will either related to the text (agree or disagree). Your task is to determine whether the headline is related or not to the text.

<text> {text} </text>

<headline> {headline} </headline>

Is the headline related or unrelated to the text? The answer is: """

In [74]:
t = time.time()
answers = {}
unique_outputs = set()
for i, d in tqdm(data.items()):
    text = d['article_body']
    answers[i] = []
    for headline, label in d['headlines']:
        p = prompt.format(text=text, headline=headline)
        response = ollama.chat(model='llama3.1', messages=[
          {
            'role': 'user',
            'content': p,
          },
        ])
        pred = response['message']['content']
        answers[i].append((pred, label))
        unique_outputs.add(pred)
    
    if len(answers) == 10:
        break
    
    
print("Elapsed time: ", time.time() - t)

  0%|          | 0/1683 [00:00<?, ?it/s]

Elapsed time:  472.87218618392944


In [78]:
acc = []
extra = []
for i, dd in answers.items():
    for d in dd:
        pred, label = d
        label = label if label == "unrelated" else "related"
        pred = "unrelated" if "unrelated" in pred else "related"
        print(pred, label)
        acc.append(int(pred == label))
        if pred not in ['related', 'unrelated']:
            extra.append(pred)

print('Average acc:', sum(acc)/len(acc))
print(extra)

unrelated unrelated
unrelated unrelated
related unrelated
related unrelated
related unrelated
related unrelated
related unrelated
related unrelated
related unrelated
related unrelated
related unrelated
unrelated unrelated
related unrelated
related unrelated
related unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
related unrelated
related unrelated
unrelated unrelated
related unrelated
unrelated unrelated
related unrelated
related related
unrelated unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
related unrelated
unrelated unrelated
related unrelated
unrelated unrelated
unrelated unrelated
related related
unrelated unrelated
related unrelated
related unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
related unrelated
unrelated unrelated
related unrelated
unrelated unrelated
unrelated unrelated
unrelated unrelated
related unre